<a href="https://colab.research.google.com/github/nisarahamedk/Kaggle_Notebooks/blob/master/notebooks/exp2_lstm_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LSTM Training on Raw Crypto Data

In [4]:
## Setting up fastai and PyTorch
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   665  100   665    0     0   3228      0 --:--:-- --:--:-- --:--:--  3228
    100% |████████████████████████████████| 5.7MB 7.0MB/s 
  Found existing installation: Pillow 5.4.0
    Uninstalling Pillow-5.4.0:
      Successfully uninstalled Pillow-5.4.0
Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 23kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61fe2000 @  0x7faf14e742a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
ln: failed to create symbolic link '/content/data': File exists
Cloning into 'course-v3'...
remote: E

In [5]:
## Creating directories to hold the data
!mkdir -p data/crypto

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:74: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
[autoreload of PIL.Image failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: The _imaging extension was built for another version of Pillow or PIL
]


In [0]:
## Upload the data using colab uploaded

In [0]:
%%capture
!cd data/crypto && unzip btc_baseline.zip

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# imports
from fastai.tabular import *
from fastai.text import *
from pathlib import Path

#### Load the data and Preprocess

In [0]:
path = Path('.')
data_path = path/'data/crypto'

In [4]:
df = pd.read_csv(data_path/'btc_baseline.csv')
df.head()

,timestamp,amount,price
0,1542301200363,0.006175,5684.0
1,1542301200376,0.005991,5684.0
2,1542301200672,0.002000,5683.9
3,1542301200710,0.264759,5685.4
4,1542301200710,0.135241,5683.9


We need to comeup with a target variable which will be used for training. That is whether it is *buy, sell or hold*

**Target based on 1000 timesteps ahead price**

In [0]:
def raw_data_for_rnn_regression(df, label_t=1000, cls_thresh=0.02):
    """
    function that adds a target variable, BUY, SELL or HOLD. based on the timestep it should look into future and 
    the threshold of price pct_change
    :param df: 
    :param label_t: 
    :param cls_thresh: 
    :return: 
    """
    df = pd.DataFrame(df)
    # remove all columns except timestamp, amount, price
    columns = ['timestamp', 'amount', 'price']
    for col in df.columns:
        if col not in columns:
            df.drop(col, axis=1, inplace=True)

    # set index timestamp
    df.set_index('timestamp', inplace=True)

    # Adding labels 

    df['target'] = df['price'].shift(-1)
    df.dropna(inplace=True)

    return df


In [32]:
df = pd.read_csv(data_path/'btc_baseline.csv')
df.head()

main_df = raw_data_for_rnn_regression(df)
main_df.head()

,amount,price,target
timestamp,,,
1542301200363,0.006175,5684.0,5684.0
1542301200376,0.005991,5684.0,5683.9
1542301200672,0.002000,5683.9,5685.4
1542301200710,0.264759,5685.4,5683.9
1542301200710,0.135241,5683.9,5683.8


In [33]:
main_df.reset_index(inplace=True)
main_df.drop('timestamp', axis=1, inplace=True)
main_df.head()

,amount,price,target
0,0.006175,5684.0,5684.0
1,0.005991,5684.0,5683.9
2,0.002000,5683.9,5685.4
3,0.264759,5685.4,5683.9
4,0.135241,5683.9,5683.8


**Creating Train and Valid Set** 

In [0]:
times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

valid_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
train_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%

In [35]:
print(len(train_df))
print(len(valid_df))

3232569
170135


In [52]:
# Normalize train and valid set
cat_names = []
cont_names = ['amount', 'price', 'target']
tfm = Normalize(cat_names, cont_names)
tfm(train_df)
tfm(valid_df, test=True)

train_df.head(), valid_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


(     amount     price    target
 0  0.013168  2.960663  2.960668
 1  0.013104  2.960663  2.960484
 2  0.011723  2.960478  2.963255
 3  0.102642  2.963250  2.960484
 4  0.057827  2.960478  2.960299,            amount     price    target
 3232569 -0.083262 -0.401553 -0.401552
 3232570  0.008059 -0.401553 -0.401552
 3232571 -1.156718 -0.401553 -0.403585
 3232572  0.009301 -0.403585 -0.407280
 3232573  0.004235 -0.407281 -0.415041)

In [53]:
full_df = pd.concat([train_df, valid_df])
full_df.head()

,amount,price,target
0,0.013168,2.960663,2.960668
1,0.013104,2.960663,2.960484
2,0.011723,2.960478,2.963255
3,0.102642,2.963250,2.960484
4,0.057827,2.960478,2.960299


**Pre-process the data**

In [0]:
def get_batches(arr_x, arr_y, batch_size, seq_length):
  
  total_vals_in_a_batch = batch_size * seq_length
  
  num_batches = len(arr_x) // total_vals_in_a_batch
  # print(f'Number of batches: {num_batches}')
  
  # keep only enough examples to form full batchs
  arr_x = arr_x[:num_batches * total_vals_in_a_batch, :]
  arr_y = arr_y[:num_batches * total_vals_in_a_batch, :]
  
  # reshape to batch_size x (seq_length * num_batches)
  # so that every batch we can slide over and get next set of sequence in order
  arr_x = arr_x.reshape(batch_size, -1, 2)
  # print(f'arr_x shape: {arr_x.shape}')
  arr_y = arr_y.reshape(batch_size, -1)
  # print(f'arr_y_shape: {arr_y.shape}')
  
  # iterate through the array 1 sequence length at a time
  for n in range(0, arr_x.shape[1], seq_length):
    X = arr_x[:, n : n + seq_length]
    y = arr_y[:, n : n + seq_length]
    
    yield X, y

In [39]:
train_arr_x = train_df.values[:, 0: 2]
train_arr_x.shape

(3232569, 2)

In [40]:
train_arr_y = train_df.values[:, 2]
train_arr_y.shape

(3232569,)

In [0]:
full_arr_x = full_df.values[:, 0:2]
full_arr_x = full_arr_x.astype('float32')
full_arr_y = full_df.values[:, 2]
full_arr_y = full_arr_y.astype('float32')
full_arr_y = np.expand_dims(full_arr_y, axis=1)

In [55]:
batches = get_batches(full_arr_x, full_arr_y, batch_size=60, seq_length=1000)
next(batches)

(array([[[ 1.316758e-02,  2.960663e+00],
         [ 1.310357e-02,  2.960663e+00],
         [ 1.172278e-02,  2.960478e+00],
         [ 1.026424e-01,  2.963250e+00],
         ...,
         [-1.963008e-01,  3.069128e+00],
         [-2.354139e-02,  3.069312e+00],
         [-5.808556e-02,  3.067280e+00],
         [-2.329357e-02,  3.062291e+00]],
 
        [[ 1.204845e-02,  2.868274e+00],
         [ 4.563267e-02,  2.868274e+00],
         [ 4.563267e-02,  2.868274e+00],
         [ 4.563267e-02,  2.868274e+00],
         ...,
         [-1.619789e-01,  2.820232e+00],
         [ 1.424349e-02,  2.820417e+00],
         [ 1.229452e-02,  2.820417e+00],
         [ 1.004445e-02,  2.820232e+00]],
 
        [[-6.270216e-03,  2.871970e+00],
         [-6.270216e-03,  2.871970e+00],
         [-6.270216e-03,  2.871970e+00],
         [-6.270216e-03,  2.871970e+00],
         ...,
         [-4.210087e-03,  2.790852e+00],
         [ 8.426874e-03,  2.790852e+00],
         [ 1.102354e-02,  2.790852e+00],
         

#### Define the Network

In [46]:
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print('GPU Available')
else:
  print('No GPU, Will be training on CPU')

GPU Available


**Model Definition**

In [0]:
class VanillaLSTM(nn.Module):
  
  def __init__(self, n_features, n_hidden=256, n_layers=2, drop_prob=0.2, lr=0.001):
    
    super().__init__()
    self.n_features = n_features
    self.drop_prob = drop_prob
    self.n_layers = n_layers
    self.n_hidden = n_hidden
    self.lr = lr
    
    # the LSTM layer
    self.lstm = nn.LSTM(input_size=n_features, hidden_size=self.n_hidden, num_layers=self.n_layers, dropout=self.drop_prob, batch_first=True)
    # Additional Dropout
    self.dropout = nn.Dropout(self.drop_prob)
    # Final FC layer, One hot encoded classes, BUY, SELL, HOLD
    self.fc = nn.Linear(n_hidden, 1)
    
  def forward(self, x, hidden):
    
    # Get the output and new hidden state form LSTM
    ### Debug Prints
#     print(f'x on {x.device}')
#     print(f'hidden on {hidden[0].device}')
#     print(f'x shape {x.shape}')
#     print(f'hidden shape {hidden[0].shape}')
#     print(f'X type{x.type()}')
#     print(f'hidden type{hidden[0].type()}')
    r_output, hidden = self.lstm(x, hidden)
    
    # pass through a dropout layer
    out = self.dropout(r_output[-1])
    
    # LSTM output will have shape batch_size x seq_length x n_hidden
    # we need to stack em up to pass it to the fully connected
    out = out.contiguous().view(-1, self.n_hidden)
    
    # pass through the FC
    out = self.fc(out)
    
    # return the final output and hidden state
    return out, hidden
  
  def init_hidden(self, batch_size):
    """
    initial hidden state
    """
    
    weight = next(self.parameters()).data
    
    # the tuple represents the Long term and short term memory state of th LSTM
    if(train_on_gpu):
      hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(), 
               weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
    else:
      hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(), 
               weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
    return hidden
    

In [0]:
def train(net, data_x, data_y, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
  
  net.train() # puttin the model in training mode.
  
  opt = torch.optim.Adam(net.parameters(), lr=lr)
  criterion = nn.MSELoss()
  
  # create training and validation set
  val_idx = int(len(data_x) * (1 - val_frac))
  data_x, val_data_x = data_x[:val_idx], data_x[val_idx:]
  data_y, val_data_y = data_y[:val_idx], data_y[val_idx:]
  
  if(train_on_gpu):
    net.cuda()
    
  counter = 0
  n_features = net.n_features
  
  for e in range(epochs):
    
    # initialize the hidden state of the network.
    h = net.init_hidden(batch_size)
    
    for x, y, in get_batches(data_x, data_y, batch_size, seq_length):
      
      counter += 1
      
      # convert to torch tensors
      inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
      # put it on to GPU if available
      if train_on_gpu:
        inputs, targets = inputs.cuda(), targets.cuda()
      
      
      # creating a new variable for the hidden state
      # otherwise, we'd backprop through the entire history
      h = ([each.data for each in h])
      
      # Zero accumulated gradients
      net.zero_grad()
      
      # get the output of the model
      outputs, h = net(inputs, h)
      
      ### Debug Prints
#       print(f'outputs shape: {outputs.shape}')
#       print(f'outputs type: {outputs.dtype}')
#       print(outputs)
#       print(f'targets shape: {targets.shape}')
#       print(f'targets type: {targets.dtype}')
#       print(targets)
      
      
      # calculate the loss and perform backprop
      loss = criterion(outputs, targets.view(batch_size * seq_length))
      
      loss.backward()
      
      # clip the gradients to avoid explosion
      nn.utils.clip_grad_norm_(net.parameters(), clip)
      
      opt.step()
      
      # print stats
      if counter % print_every == 0:
        # print(f'Validation, epoch: {counter}')
        # Get validation loss
        val_h = net.init_hidden(batch_size)
        
        val_losses = []
        net.eval()
        
        for x, y in get_batches(val_data_x, val_data_y, batch_size, seq_length):
          
          # convert to torch tensors
          x, y = torch.from_numpy(x), torch.from_numpy(y)
          
          # creating new variable for the hidden state
          val_h = ([each.data for each in val_h])
          
          inputs, targets = x, y
          if train_on_gpu:
            inputs, targets = inputs.cuda(), targets.cuda()
          
          output, val_h = net(inputs, val_h)
          
          val_loss = criterion(output, targets.view(batch_size * seq_length))
          
          val_losses.append(val_loss.item())
          
          # print(f'Validation, epoch: {counter}')
          
        net.train()
        
        print(f'Training Loss: {loss.item()}')
        print(f'Validation Loss: {sum(val_losses) / len(val_losses)}')

#### Instantiating the model and Training

In [68]:
n_hidden = 1000
n_layers = 2

net = VanillaLSTM(n_features=2, n_hidden=n_hidden, n_layers=n_layers, drop_prob=0.2, lr=0.01)
net

VanillaLSTM(
  (lstm): LSTM(2, 1000, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=1000, out_features=1, bias=True)
)

In [0]:
batch_size = 30
seq_length = 60
epochs = 20

In [70]:
train(net, full_arr_x, full_arr_y, epochs=epochs, batch_size=batch_size, seq_length=seq_length, lr=0.01, print_every=10)

Training Loss: 2.321678400039673
Validation Loss: 0.03383177913055218
Training Loss: 1.5044082403182983
Validation Loss: 1.8738104054536768
Training Loss: 2.1106483936309814
Validation Loss: 0.6521342163363462
Training Loss: 1.5973299741744995
Validation Loss: 0.2744826959869849
Training Loss: 1.920238733291626
Validation Loss: 0.04779738422345232
Training Loss: 1.6198408603668213
Validation Loss: 1.9725683435561165
Training Loss: 2.0517654418945312
Validation Loss: 0.6531345282912885
Training Loss: 1.5429884195327759
Validation Loss: 0.23094485266498788
Training Loss: 1.8946977853775024
Validation Loss: 0.04785558786341753
Training Loss: 1.6289088726043701
Validation Loss: 1.8917193242481776
Training Loss: 1.9327667951583862
Validation Loss: 0.6243925454124571
Training Loss: 1.4409536123275757
Validation Loss: 0.23430687433512754
Training Loss: 1.8136335611343384
Validation Loss: 0.04652392876053613
Training Loss: 1.568530559539795
Validation Loss: 1.8664488186911932
Training Loss: 1.

KeyboardInterrupt: ignored

In [0]:
net = None